In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import numpy
import time 

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score


In [3]:
#Load some Data
data = pd.read_csv('wbcd.data',header=None) 
labels = data.iloc[:, -1]
data = data.drop(data.columns[[-1]],axis = 1)

labels.head()

In [7]:
#Load some Data
wbcd_data = pd.read_csv('../input/aiml-426-p2-data/wbcd.data',header=None)

In [ ]:
#Deap Params

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", random.randint, 0, 1)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 30)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)# Creator
creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.Fitness)

In [ ]:
#Evaluation Function
def wrappereval(individual):
    temp_features = []
    index = 0
    feature_count = 0
    for indicator in individual:
        if indicator == 1:
            temp_features.append(index)
            feature_count += 1
        index += 1
        
    if feature_count == 0:
        return 0,
    temp_df = data.iloc[:, temp_features]
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(temp_df, labels)
    preds = knn.predict(temp_df)
    score = accuracy_score(labels, preds) 
    
    return score,

In [ ]:
#Register these above functions in DEAP toolbox
toolbox.register("evaluate", wrappereval)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb = 0.05)
toolbox.register("select", tools.selTournament, tournsize=5)

In [ ]:
def mainwrapper():
    NGEN = 20
    MU = 100
    LAMBDA = 95
    CXPB = 0.85
    MUTPB = 0.15

    pop = toolbox.population(n=MU)
    hof = tools.HallOfFame(maxsize = 1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)

    tic = time.clock()
    algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof)
    toc = time.clock()
    comptime = toc - tic
    
    return pop, stats, hof, comptime

In [ ]:
def evalhof(winner):
    features = []
    index = 0
    for indicator in winner:
        if indicator == 1:
            features.append(index)
        index += 1
    
    temp_df = data.iloc[:, features]
    nb = GaussianNB()
    nb.fit(temp_df, labels)
    preds = nb.predict(temp_df)
    score = accuracy_score(labels, preds) 
    return score


In [ ]:
wrapper_scores = []
for i in range(5):
    wpop, wstats, whof, wtime = mainwrapper()
    wrapper_scores.append(evalhof(whof[0]))
    

In [ ]:
import statistics

stnd_dev = statistics.stdev(wrapper_scores)
mean = statistics.mean(wrapper_scores)

print(stnd_dev)
print(mean)